In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
import openpyxl
import pickle

from helper_functions import *

In [2]:
#Imports parameters from pickle file
prms = load_prms()

#The scalars for the system are also embedded in this prms object, but getting them directly from there can ba a bit cumbersome. So we extract them here for convenience.
sclrs = get_scalars(prms)

Successfully assigned value to t_max as [6.]
Successfully assigned value to g as [0.001]
Successfully assigned value to h_w as [102375.]
Successfully assigned value to h_s as [43875.]
Successfully assigned value to l_b as [375000.]
Successfully assigned value to gamma as [80.]
Successfully assigned value to p_bse as [0.157]
Successfully assigned value to i as [500000.]
Successfully assigned value to w as [50.]
Successfully assigned value to lf as [0.85]
Successfully assigned value to rm as [0.15]


In [3]:
print(f"Checking Constraint for Effective Miles Traveled: ")

x_test = np.sum(prms["T"], axis=0)

if np.all(x_test > np.squeeze(prms["x"])):
    print("TRUE \n")
else:
    print("FALSE! Change T or X\n")

Checking Constraint for Effective Miles Traveled: 


ValueError: operands could not be broadcast together with shapes (4,) (4,10000) 

In [5]:
print(f"Average number of trips per mode, bucket")
trips = np.zeros(prms["T"].shape)
for i in range (trips.shape[1]):
    trips[:,i] = prms["T"][:,i] / prms["d^e"][i]
print(f"{trips}\n")

print("Estimating Average number of effective miles traveled per mode, bucket")
T_actual = np.multiply(trips, prms["d"])
print(T_actual)

print("Mutual congestion effects")
Total_mode_actual = np.sum(T_actual, axis = 1)
congestion_effects = np.zeros_like(Total_mode_actual)
for i_1 in range (T_actual.shape[0]):
    c_e_counter = 0
    for i_2 in range (T_actual.shape[0]):
        c_e_counter += prms["c"][i_1, i_2] * Total_mode_actual[i_2]
    congestion_effects[i_1] = c_e_counter
congestion_effects = congestion_effects * sclrs["g"]
print(congestion_effects)

print("Real Velocity of Travelers")
v_real = prms["v_max"] * np.exp(-1 * congestion_effects.reshape(prms["v_max"].shape[0],1))
print(f"{v_real}\n")

print("Real Travel Times")
t_travel = T_actual / v_real
print(f"{t_travel}\n")

print("Real Added Wait Time")
t_wait = trips * prms["t^t"]
print(f"{t_wait}\n")

print("Total Time per Mode-Bucket")
t_total = t_travel + t_wait
print(f"{t_total}\n")

print(f"Total per Person Travel Time: {np.sum(t_total)} hrs")

print("Checking Total Time Constraint:\n")
if(np.sum(t_total) > sclrs["t_max"]):
    print(f"FALSE. Travel less, or reconsider parameters.\n")
else:
    print("TRUE\n")

Average number of trips per mode, bucket
[[1.40000000e+00 5.60000000e-01 5.60000000e-01 3.33333333e-01]
 [2.00000000e-01 3.60000000e-01 1.20000000e-01 6.66666667e-02]
 [1.00000000e-01 8.00000000e-02 1.20000000e-01 3.33333333e-02]
 [1.40000000e-01 4.00000000e-01 2.40000000e-01 6.66666667e-02]
 [1.40000000e-01 4.00000000e-01 2.40000000e-01 6.66666667e-02]
 [1.80000000e+00 1.00000000e-01 8.00000000e-07 1.66666667e-08]
 [1.00000000e+00 3.00000000e-01 2.40000000e-01 5.00000000e-02]]

Estimating Average number of effective miles traveled per mode, bucket
[[1.225e+00 2.450e+00 1.575e+01 4.000e+01]
 [1.750e-01 1.575e+00 3.375e+00 8.000e+00]
 [8.750e-02 3.500e-01 3.375e+00 4.000e+00]
 [2.100e-01 3.000e+00 9.000e+00 1.000e+01]
 [2.100e-01 2.500e+00 7.500e+00 1.000e+01]
 [9.000e-01 2.500e-01 1.000e-05 1.000e-06]
 [5.000e-01 7.500e-01 3.000e+00 3.000e+00]]
Mutual congestion effects
[0.07778475 0.07778475 0.02264725 0.06947163 0.011885   0.
 0.0059425 ]
Real Velocity of Travelers
[[23.12908865 27.7

In [32]:
#How we understand when vehicles will be charging
def get_time_charge(i):
    result_matrix = np.ones_like(t_b_electric)

    result_matrix[(i-t_b_electric) % 24 >= (t_e_electric - t_b_electric) % 24] = 0

    return result_matrix * load_rates

print("System Maximum Capacity")
l_max = sclrs["l_b"] * 1/sclrs["lf"] * (1 + sclrs["rm"])
print(f"{l_max}\n")

print("Lowest Renewable Available")
renewable_avail = prms["r"][:,1] * sclrs["h_s"] + prms["r"][:,0] * sclrs["h_w"]
r_min = min(renewable_avail)
print(f"{r_min}\n")

print("Normal Dispatchable Expectation")
dispatch_max = l_max - r_min
print(f"{dispatch_max}\n")

print("Electricity Expended / Person")
indeces_electric = [1,4]
T_actual_electric = T_actual[indeces_electric, :]
e_electric = prms["e"][indeces_electric, :]
electricity_expended = T_actual_electric / e_electric
print(f"{electricity_expended}\n")

print("kWh / hr load  / Person")
t_b_electric = prms["t^b"][indeces_electric,:]
t_e_electric = prms["t^e"][indeces_electric,:]
charge_intervals = (t_e_electric - t_b_electric) % 24
load_rates_normal = electricity_expended / charge_intervals
print(f"{load_rates_normal}\n")

print("kWh / hr load")
load_rates = load_rates_normal * sclrs["i"]
print(f"{load_rates}\n")

print("Baseline Load (kW)")
baseline_load = sclrs["l_b"] * prms["l_b^t"]
print(f"{baseline_load}\n")

print("Total Load (kW)")
time_block = np.zeros((t_b_electric.shape[0], t_b_electric.shape[1], 24), dtype=int)
total_load = np.zeros_like(baseline_load)
for k in np.arange(24):
    time_block[:, :, k] = get_time_charge(k)
    total_load[k] = baseline_load[k] + np.sum(time_block[:,:,k])
print(f"{total_load}\n")

print("Max Load Required by Dispatch Resources (kW)")
max_dispatchable_load = np.max(np.squeeze(total_load) - renewable_avail)
print(f"{max_dispatchable_load}\n")

print("Checking Max Load Constraint (kW)")
if(max_dispatchable_load > dispatch_max):
    print(f"FALSE. Travel less, or reconsider parameters.\n")
else:
    print("TRUE\n")





System Maximum Capacity
[507352.94117647]

Lowest Renewable Available
71662.49999999999

Normal Dispatchable Expectation
[435690.44117647]

Electricity Expended / Person
[[0.03888889 0.315      0.5625     1.14285714]
 [0.         0.075      0.225      0.41      ]]

kWh / hr load  / Person
[[0.00277778 0.02423077 0.05113636 0.11428571]
 [0.         0.00357143 0.0140625  0.041     ]]

kWh / hr load
[[ 1388.88888889 12115.38461538 25568.18181818 57142.85714286]
 [    0.          1785.71428571  7031.25       20500.        ]]

Baseline Load (kW)
[[375000.]
 [337500.]
 [337500.]
 [337500.]
 [300000.]
 [337500.]
 [337500.]
 [412500.]
 [412500.]
 [412500.]
 [412500.]
 [412500.]
 [412500.]
 [412500.]
 [375000.]
 [375000.]
 [375000.]
 [375000.]
 [375000.]
 [375000.]
 [375000.]
 [375000.]
 [375000.]
 [375000.]]

Total Load (kW)
[[472998.]
 [435498.]
 [433713.]
 [433713.]
 [396213.]
 [435498.]
 [378356.]
 [427788.]
 [422704.]
 [441815.]
 [441815.]
 [441815.]
 [441815.]
 [441815.]
 [404315.]
 [4043

alpha dictionary:

0: Trips

1: Time

2: Fuel

3: Effort

4: Electricity

5: Safety

In [33]:
#Calculate Utility from Extra Trips Added:

U = np.zeros([6])

print("Number of extra trips, per bucket:")
extra_miles = np.sum(prms["T"], axis=0) - np.squeeze(prms["x"])
extra_trips = extra_miles / np.squeeze(prms["d^e"])
print(f"{extra_trips}\n")

print("Utility from extra trips:")
U[0] = prms["alpha"][0] * np.sum(np.log(extra_trips + 1))
print(U[0])


Number of extra trips, per bucket:
[2.5        1.         0.72       0.28333333]

Utility from extra trips:
410.65429492683876


C:\Users\cjgir\AppData\Local\Temp\ipykernel_20672\4091707781.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  U[0] = prms["alpha"][0] * np.sum(np.log(extra_trips + 1))


In [34]:
#Calculate Utility from Time Spent:

print("Utility from Time Cost:")
U[1] = prms["alpha"][1] * np.sum(t_total)
print(U[1])

Utility from Time Cost:
-130.0312478068984


C:\Users\cjgir\AppData\Local\Temp\ipykernel_20672\1492204694.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  U[1] = prms["alpha"][1] * np.sum(t_total)


In [35]:
#Calculate Utility from Fuel Cost:

fuel_indeces = [0,2,3]
f_fuel = prms["f"][fuel_indeces, :]
T_actual_fuel = T_actual[fuel_indeces, :]

print("Fuel Expended: gal / Person:")
fuel_expended = T_actual_fuel/f_fuel
print(f"{fuel_expended}\n")

print("Utility from Fuel:")
U[2] = prms["alpha"][2] * np.sum(fuel_expended)
print(U[2])

Fuel Expended: gal / Person:
[[0.037975   0.06604348 0.3906     0.95384615]
 [0.00120556 0.00434    0.03948113 0.04509091]
 [0.         0.0155     0.03985714 0.03875   ]]

Utility from Fuel:
-16.326893716861026


C:\Users\cjgir\AppData\Local\Temp\ipykernel_20672\522320741.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  U[2] = prms["alpha"][2] * np.sum(fuel_expended)


In [36]:
#Calculate Utility from Effort

# print(T_actual)
# print(np.pow(prms["d"],2))

print("Utility from Effort:")
effort_matrix = np.multiply(trips, np.pow(prms["d"],2))
U[3] = prms["alpha"][3] * np.sum(np.multiply(effort_matrix, prms["b"]))
print(f"{U[3]}\n")

print(np.multiply(effort_matrix, prms["b"]))

Utility from Effort:
-29.355010937499998

[[1.0718750e-03 1.0718750e-02 4.4296875e-01 4.8000000e+00]
 [1.5312500e-04 6.8906250e-03 9.4921875e-02 9.6000000e-01]
 [7.6562500e-04 1.5312500e-02 9.4921875e-01 4.8000000e+00]
 [0.0000000e+00 1.8000000e-02 2.7000000e-01 1.2000000e+00]
 [0.0000000e+00 1.2500000e-03 1.8750000e-02 1.2000000e-01]
 [4.5000000e-01 6.2500000e-01 0.0000000e+00 0.0000000e+00]
 [5.0000000e-02 3.7500000e-01 7.5000000e+00 3.6000000e+01]]


C:\Users\cjgir\AppData\Local\Temp\ipykernel_20672\2486519971.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  U[3] = prms["alpha"][3] * np.sum(np.multiply(effort_matrix, prms["b"]))


In [37]:
# Electricity Cost

print("Load Expected to be Dispatched to Thermal (kW):")
dispatchable_load = np.squeeze(total_load) - renewable_avail
print(f"{dispatchable_load}\n")

print("Expected Dispatch to Thermal Load Over Minimum (kW):")
min_dispatch = min(dispatchable_load)
dispatch_excess = dispatchable_load - min_dispatch
print(f"{dispatch_excess}\n")

print("Marginal Effects on Price:")
min_baseline_load = min(baseline_load)
marginal_effects = sclrs["gamma"] * np.pow(dispatch_excess / min_baseline_load, 2)
print(f"{marginal_effects}\n")

print("Baseline Prices:")
print(f"{sclrs["p_bse"] * np.squeeze(prms["p_b^t"])}\n")

print("Adjusted Price for Marginal Impact:")
real_prices = sclrs["p_bse"] * (np.squeeze(prms["p_b^t"]) + marginal_effects)
print(f"{real_prices}\n")

print("Utility from Cost:")
cost_count_electric = 0
for i in range(24):
    cost_count_electric += np.sum(time_block[:,:,i] / sclrs["i"] * real_prices[i])
U[4] = prms["alpha"][4] * cost_count_electric
print(f"{U[4]}\n")



#dynamic_electricity_price = sclrs["gamma"] * np.pow(dispatchable_load, 2) * 

#print(dynamic_electricity_price)

Load Expected to be Dispatched to Thermal (kW):
[401335.5 363835.5 351813.  351813.  304075.5 325810.5 259893.5 268375.5
 235504.  219515.  220977.5 188802.5 197577.5 210740.  190790.  206877.5
 224427.5 253677.5 262452.5 284706.5 377654.  387891.5 387891.5 398129. ]

Expected Dispatch to Thermal Load Over Minimum (kW):
[212533.  175033.  163010.5 163010.5 115273.  137008.   71091.   79573.
  46701.5  30712.5  32175.       0.    8775.   21937.5   1987.5  18075.
  35625.   64875.   73650.   95904.  188851.5 199089.  199089.  209326.5]

Marginal Effects on Price:
[4.01513565e+01 2.72324899e+01 2.36199317e+01 2.36199317e+01
 1.18114351e+01 1.66855041e+01 4.49238247e+00 5.62832207e+00
 1.93869342e+00 8.38451250e-01 9.20205000e-01 0.00000000e+00
 6.84450000e-02 4.27781250e-01 3.51125000e-03 2.90405000e-01
 1.12812500e+00 3.74112500e+00 4.82162000e+00 8.17562419e+00
 3.17021236e+01 3.52323822e+01 3.52323822e+01 3.89489632e+01]

Baseline Prices:
[0.0785 0.0628 0.0785 0.0314 0.0314 0.0314 0.04

C:\Users\cjgir\AppData\Local\Temp\ipykernel_20672\3999311481.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  U[4] = prms["alpha"][4] * cost_count_electric


In [38]:
# Calculate Safety Impacts

print("Utility from Safety Effects:")
U[5] = prms["alpha"][5] * np.sum(np.multiply(prms["nu"], T_actual) * sclrs["w"])
print(f"{U[5]}\n")

Utility from Safety Effects:
-29.032



C:\Users\cjgir\AppData\Local\Temp\ipykernel_20672\2207670311.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  U[5] = prms["alpha"][5] * np.sum(np.multiply(prms["nu"], T_actual) * sclrs["w"])


In [39]:
# Finally, What's the final utility!?

print("Final Utility Value:")
maximize_me = np.sum(U)
print(maximize_me)

Final Utility Value:
196.1916764662507
